# 演習問題

与えられたニュースをカテゴリに仕分けする分類器（カテゴリ分類器）を構築したい．今回は，[livedoor ニュースコーパス](https://www.rondhuit.com/download.html)を用い，記事の本文やタイトルからその情報源を推定する分類器を構築する．

+ トピックニュース: http://news.livedoor.com/category/vender/news/
+ Sports Watch: http://news.livedoor.com/category/vender/208/
+ ITライフハック: http://news.livedoor.com/category/vender/223/
+ 家電チャンネル: http://news.livedoor.com/category/vender/kadench/
+ MOVIE ENTER: http://news.livedoor.com/category/vender/movie_enter/
+ 独女通信: http://news.livedoor.com/category/vender/90/
+ エスマックス: http://news.livedoor.com/category/vender/smax/
+ livedoor HOMME: http://news.livedoor.com/category/vender/homme/
+ Peachy: http://news.livedoor.com/category/vender/ldgirls/


なお，ニュースのカテゴリ分類は[ニュース・キュレーションサービスなどで実際に用いられている](https://webtan.impress.co.jp/e/2015/04/14/19666)技術である．今回の演習では，構築する分類器は線形識別器（多クラスロジスティック回帰など）に限定する（多層ニューラルネットワークや非線形サポートベクトルマシンを使ってはいけない）．

## 1. データのダウンロードと整形

[livedoor ニュースコーパス](https://www.rondhuit.com/download.html)は[クリエイティブ・コモンズライセンス「表示 – 改変禁止」](https://creativecommons.org/licenses/by-nd/2.1/jp/)のライセンスで配布されているため，データを加工したものを再配布することができない．そこで，データのダウンロードから整形まで，各自の環境で実行する必要がある．データの整形を行う手順を以下に示すので，そのまま実行するだけでよい．ただし，この演習では全員が同じ学習データ，検証データ，評価データを用いたいので，以下の手順を改変することなく実行せよ．

### 1.1. 訓練データ，検証データ，評価データの準備

コーパスをダウンロード．

In [1]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2020-12-26 12:29:57--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
CA証明書 '/etc/ssl/certs/ca-certificates.crt' をロードしました
www.rondhuit.com (www.rondhuit.com) をDNSに問いあわせています... 59.106.19.174
www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 8855190 (8.4M) [application/x-gzip]
`ldcc-20140209.tar.gz' に保存中

ldcc-20140209.tar.g 100%[===================>]   8.44M  32.0MB/s 時間 0.3s       

2020-12-26 12:29:57 (32.0 MB/s) - `ldcc-20140209.tar.gz' へ保存完了 [8855190/8855190]



ダウンロードしたコーパスを解凍．

In [2]:
!tar -zxvf ldcc-20140209.tar.gz > /dev/null

解凍したファイルを読み込み，記事のリストとしてデータ`D`を作成する．

In [3]:
import pathlib

D = []
p = pathlib.Path('text')
for d in p.iterdir():
    if not d.is_dir():
        continue
    source = d.name
    for fname in d.glob('*.txt'):
        with open(fname) as fi:
            url = fi.readline().strip()
            timestamp = fi.readline().strip()
            title = fi.readline().strip()
            text = [line.strip() for line in fi if line.strip()]
            D.append(
                dict(source=source, url=url, timestamp=timestamp, title=title, text=text)
                )

訓練データ`Dtrain`, 検証データ`Ddev`，評価データ`Dtest`に分ける．

In [4]:
D.sort(key=lambda x: x['url'])

Dtrain = []
Ddev = []
Dtest = []

for i, d in enumerate(D):
    if i % 10 == 8:
        Ddev.append(d)
    elif i % 10 == 9:
        Dtest.append(d)
    else:
        Dtrain.append(d)

コーパスを訓練データ，検証データ，評価データに正しく分割できたかを，ハッシュ値を用いてチェックする．もし，以下のコードを実行した時に**AssertionError例外が出た場合はそれまでの手順を変更してしまった可能性がある**（正常であれば"OK"と表示される）．どうしてもAssertionError例外が出る場合は連絡すること．

In [35]:
import hashlib

def compute_hash(D):
    m = hashlib.sha256()
    for d in D:
        m.update(d['url'].encode('utf-8'))
    return m.hexdigest()

assert compute_hash(Dtrain) == 'f1294a19b25952e5b18510e3eb74c21be9d5d18a86c369d2d2639c9e5ea93d6c'
assert compute_hash(Ddev) == '64f709e1e739ac880b8b7acc49ce342b60e80b804279bac68c5f27d08b5fb141'
assert compute_hash(Dtest) == '4acf6822099a9e4cc5794cade26ae0ddd8df88ccc99690e7b48cdd8aa3bf1bcd'
print("OK")

OK


### 1.2 日本語の分かち書き

タイトルと本文の日本語を分かち書きするために，[MeCab](https://taku910.github.io/mecab/)をインストールする（mecab-python3の最新版は環境によってエラーが出るので，v0.996.5を指定してインストールしている）．

In [7]:
!pip install mecab-python3==0.996.5

     |████████████████████████████████| 17.1 MB 10.8 MB/s eta 0:00:01


タイトルと本文を単語（形態素）区切りで分割する．

In [9]:
import MeCab
tagger = MeCab.Tagger('-Owakati')
def tokenize(s):
    return tagger.parse(s).split()

def add_tokenization(D):
    for d in D:
        d['title.tokenized'] = tokenize(d['title'])
        d['text.tokenized'] = [tokenize(s) for s in d['text']]

add_tokenization(Dtrain)
add_tokenization(Ddev)
add_tokenization(Dtest)

### 整形済みのデータをファイルに保存する場合

もし，整形済みのデータをファイルに保存しておきたい場合は，以下のコードを実行して"livedoor.json"というファイルに保存する．ただし，Google Colaboratory上で実行している場合は，インスタンスが消滅すると保存したファイルも消えてしまうので，以下のいずれかで対応することになる．

1. インスタンスを新たに立ち上げた（インスタンスがリセットされた）度に，これまでの処理を再実行する
1. "livedoor.json"を自分のPCに保存しておき，インスタンスを立ち上げる毎にアップロードする
1. "livedoor.json"を自分のGoogle Driveに保存しておき，インスタンスを立ち上げたときにマウントして読み込む

In [176]:
import json

with open('livedoor.json', 'w') as fo:
    json.dump(
        dict(train=Dtrain, test=Dtest, dev=Ddev),
        fo
        )

### 保存された整形済みのデータを読み込む

In [1]:
import json

with open('livedoor.json') as fi:
    D = json.load(fi)
    
Dtrain = D['train']
Ddev = D['dev']
Dtest = D['test']

## 作成されたデータの確認

訓練データの先頭の事例を表示してみる．各訓練データは，以下のような辞書で表現される．各フィールドの意味は以下の通りである．

+ `source`: 記事のカテゴリ
+ `url`: 記事のURL
+ `timestamp`: 記事の発行日時
+ `title`: 記事のタイトル
+ `text`: 記事の本文（段落（文字列）を要素としたリスト形式
+ `title.tokenized`: 記事のタイトルをMeCabで分かち書きしたもの
+ `text.tokenized`: 記事の本文を分かち書きしたもの．段落が単語のリストとして表現され，その段落のリストを格納している

`source`フィールドのクラスを目的変数とみなし，それ以外のフィールドの情報から目的変数を予測する高性能なモデルを構築するのが，今回の演習の趣旨である．

In [18]:
Dtrain[4749]

{'source': 'peachy',
 'url': 'http://news.livedoor.com/article/detail/6684605/',
 'timestamp': '2000-06-25T15:45:00+0900',
 'title': 'キーワードは「カワイク＆賢く！」イマドキスマホ女子に人気のスマホグッズ紹介',
 'text': ['・カカオチョコレート',
  '・ウサギケース ラビットしっぽ',
  '・フォンピアス\u3000イヤホンジャックアクセサリー\u3000クマ',
  '・ブラウンポンポンゴールド スマートフォンアクセ'],
 'title.tokenized': ['キーワード',
  'は',
  '「',
  'カワイク',
  '＆',
  '賢く',
  '！',
  '」',
  'イマドキスマホ',
  '女子',
  'に',
  '人気',
  'の',
  'スマホグッズ',
  '紹介'],
 'text.tokenized': [['・', 'カカオ', 'チョコレート'],
  ['・', 'ウサギ', 'ケース', 'ラビット', 'しっぽ'],
  ['・', 'フォン', 'ピアス', 'イヤホンジャックアクセサリー', 'クマ'],
  ['・', 'ブラウンポンポンゴールド', 'スマートフォンアクセ']]}

## 2. 単語の出現頻度による特徴量ベクトル

学習データ`Dtrain`に含まれる任意の事例に対して，分かち書きされたテキスト（`text.tokenized`）に含まれる単語の出現頻度を計測し，単語から頻度への連想配列（辞書）形式のオブジェクトに格納せよ（小レポート1 7-1を参考にせよ）．例として，`Dtrain[3521]`の学習事例のテキストに対して，単語の出現頻度を計測した結果の一部を示す．

```
{'4': 1,
 '月': 2,
 '19': 1,
 '日': 1,
 '（': 3,
 '）': 3,
 'より': 1,
 ...
 '類': 1,
 'と': 1,
 'なる': 1}
```

In [3]:
from collections import defaultdict

In [4]:
def token2vec(token):
    vec = defaultdict(int)
    for sentence in token:
        for word in sentence:
                vec[word] += 1
    return vec

In [126]:
token = Dtrain[238]['text.tokenized']
token2vec(token)

defaultdict(int,
            {'4': 1,
             '月': 2,
             '19': 1,
             '日': 1,
             '（': 3,
             '）': 3,
             'より': 1,
             '、': 6,
             '神戸': 1,
             '大丸': 1,
             'インテリア': 1,
             '専門': 1,
             '館': 1,
             '「': 2,
             'ミュゼエール': 1,
             '＠': 1,
             '六甲': 1,
             'アイランド': 1,
             '」': 2,
             '2': 1,
             '階': 1,
             'で': 2,
             'は': 3,
             'ヨーロッパ': 1,
             '最大': 1,
             'の': 2,
             'フィットネスマシンメーカー': 1,
             'テクノ': 1,
             'ジム': 1,
             '社': 1,
             'パートナー': 1,
             'ショールーム': 1,
             'が': 1,
             '関西': 1,
             '初めて': 1,
             '開設': 1,
             'さ': 1,
             'れる': 1,
             '。': 2,
             '展示': 1,
             '品': 1,
             'キネシス・パーソナル': 1,
             'Vision': 1,
            

## 3. 線形分類モデルの学習

2.で構築したプログラムを使い，学習データ`Dtrain`の分かち書きされたテキスト（`text.tokenized`）に含まれる単語の頻度を特徴量ベクトル$\pmb{x}$として，目的変数（情報源である`source`フィールド）を予測する線形識別モデルを学習せよ．

ヒント
+ 線形分類モデルの実装に[sklearn.linear_model.SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)を使う場合は，単語とクラス名を自然数のID番号に変換し，学習事例を`np.array`に変換する必要がある．この変換には，小レポート1 7-2が参考になるし，[sklearn.feature_extraction.DictVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html)および[sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)を使ってもよい．
+ 特徴量の空間，すなわち線形分類モデルが扱うことのできる単語集合は訓練データ中に含まれる全ての単語とすればよい．
+ 訓練データ中には出現しなかったが，検証データや評価データのみに出現する単語がある．そのような単語（OOV: out-of-vocabulary）は無視すればよい．

In [5]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing

import numpy as np

In [68]:
def token2vec(token):
    vec = defaultdict(int)
    for sentence in token:
        for word in sentence:
                vec[word] += 1
    return vec

In [87]:
train_vec = [token2vec(data['text.tokenized']) for data in Dtrain]
dev_vec = [token2vec(data['text.tokenized']) for data in Ddev]
test_vec = [token2vec(data['text.tokenized']) for data in Dtest]

In [88]:
# word to IDVec
VX = DictVectorizer()
Xtrain = VX.fit_transform(train_vec).toarray()
Xdev = VX.transform(dev_vec).toarray()
Xtest = VX.transform(test_vec).toarray()

In [98]:
# source to ID
VY = preprocessing.LabelEncoder()
Ytrain = VY.fit_transform([data['source'] for data in Dtrain])
Ydev = VY.transform([data['source'] for data in Ddev])
Ytest = VY.transform([data['source'] for data in Dtest])

In [138]:
model = SGDClassifier(loss='log')
model.fit(Xtrain, Ytrain)

SGDClassifier(loss='log')

In [139]:
#model をpickle化
import pickle

with open('SGD_loss-log_1226.pickle', 'wb') as f:
    pickle.dump(model, f)

In [140]:
Ytrain_pred=model.predict(Xtrain)
model.score(Xtrain, Ytrain)

0.994747543205693

## 4. 検証データ上での正解率

3で学習したモデルの検証データ上での正解率を求めよ．

In [141]:
model.score(Xdev,Ydev)

0.937584803256445

## 5. 検証データ上でのマクロ平均適合率，再現率，F1スコア

3で学習したモデルの検証データ上での適合率，再現率，F1スコアを求めよ．ただし，これらの指標を求めるときは，マクロ平均を用いよ．

In [6]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [142]:
Ydev_pred=model.predict(Xdev)

In [143]:
precision_score(Ydev, Ydev_pred, average='macro')

0.9360393590673417

In [144]:
recall_score(Ydev, Ydev_pred, average='macro')

0.9283801510999806

In [145]:
f1_score(Ydev, Ydev_pred, average='macro')

0.930737047396014

## 6. 検証データ上での混同行列

3で学習したモデルの検証データ上での混同行列を求めよ．

In [7]:
from sklearn.metrics import confusion_matrix

In [146]:
confusion_matrix(Ydev, Ydev_pred)

array([[90,  0,  0,  1,  1,  2,  0,  0,  0],
       [ 0, 78,  2,  0,  1,  0,  1,  2,  0],
       [ 0,  1, 85,  0,  0,  0,  0,  1,  1],
       [ 0,  1,  4, 39,  5,  3,  1,  1,  0],
       [ 0,  0,  1,  0, 77,  2,  0,  1,  1],
       [ 2,  0,  0,  3,  0, 73,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  1, 81,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 92,  3],
       [ 0,  0,  1,  0,  0,  1,  0,  1, 76]])

## 7. 事例の分類 (*)

3で学習したモデルを用い，検証データの先頭の事例のクラスを予測し，表示せよ．

In [147]:
VY.classes_

array(['dokujo-tsushin', 'it-life-hack', 'kaden-channel',
       'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch',
       'topic-news'], dtype='<U14')

In [148]:
print(f'pred: {Ydev_pred[0]}, {VY.classes_[Ydev_pred[0]]}')
print(f'true: {Ydev[0]}, {VY.classes_[Ydev[0]]}')

pred: 5, peachy
true: 5, peachy


## 8. クラスと確率の表示 (*)

3で学習したモデルを用い，検証データの先頭の事例に対して，各クラスに分類される確率（条件付き確率）を求めよ．

In [149]:
model.predict_proba(Xdev[0:1])

array([[1.32249729e-025, 2.43165634e-186, 5.60295358e-124,
        9.06614257e-289, 1.38627054e-226, 1.00000000e+000,
        0.00000000e+000, 9.65786427e-278, 1.22413911e-109]])

## 9. 検証データをターゲットとした性能向上 (**)

カテゴリ分類器のハイパーパラメータや記事からの特徴量抽出を工夫し，検証データ上でのF1スコアが最も高くなるカテゴリ分類器を見つけよ．工夫においてどのような方針でモデルを検討・実験し，その中でどのモデルの性能が最も良かったのか，説明せよ（Markdown形式で記述せよ）．さらに，検討した中で性能が最も高いカテゴリ分類器を学習するプログラムと，その分類器の評価データ上での適合率，再現率，F1スコアを報告せよ．

+ 識別モデルの学習パラメータ（L2正則化の係数など）に加えて，記事の特徴量などにも工夫する余地がある．
+ 必要であれば1.2以降の処理を変更し，単語の分かち書きの方法を変えてもよい
+ 学習データ，検証データ，評価データの分け方を変更してはならない（1.1までの手順は変更不可）
+ 言うまでもないが，評価データでカテゴリ分類器を学習してはいけない

今回は以下のような処理を行って`SGDClassifier`によるカテゴリ分類器を学習した．
- 前処理を行う
    - 助詞や助動詞などは無視する
    - 数値以外は原形にする
    - 記事本文だけでなくタイトルも用いている（タイトルを本文の先頭に結合）
    - urlは正規表現で除去
- TFIDFVectorizerを用いて単語をベクトル化する
    - CountVectorizerなど様々な方法を試した中で最も良かった方法がTF-IDFによる方法だった
    - `max_df`や`min_df`などを指定して不要な語彙を省く
- optunaでパラメータチューニング
    - 正則化項に関するパラメータ`alpha`と`L1_ratio`についてチューニングする
    - 検証データを用いてチューニングする場合と，訓練データのみを用いて交差検証によるチューニングの両方を試した結果，前者のほうが良いスコアが出たためこちらを採用した
- 最適なパラメータでのfittingを100回施行し，最も検証データのf1スコアが高かったモデルを採用する

最も検証データが良かった結果は，`params = {'alpha': 4.2842570078130996e-07, 'l1_ratio': 0.0014625643848466556}`のとき

|       | accuracy | precision | recall | f1_score |
| ----- | -------- | --------- | ------ | ---------- |
| train | 1.0      | 1.0       | 1.0    | 1.0        |
| dev   | 0.972    | 0.971     | 0.967  | 0.968      |
| test  | 0.970    | 0.968     | 0.965  | 0.967      |

である．

また，混同行列は，

In [68]:
confusion_matrix(Ydev, Ydev_pred)

array([[90,  0,  0,  0,  1,  3,  0,  0,  0],
       [ 0, 84,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 88,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0, 47,  2,  3,  1,  0,  0],
       [ 0,  0,  0,  0, 81,  1,  0,  0,  0],
       [ 0,  0,  0,  1,  2, 75,  0,  0,  0],
       [ 0,  0,  2,  0,  0,  0, 81,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 95,  0],
       [ 0,  0,  0,  1,  0,  1,  0,  2, 75]])

In [107]:
print(f'precision:{precision_score(Ydev, Ydev_pred, average=None)}')
print(f'recall_score:{recall_score(Ydev, Ydev_pred, average=None)}')
print(f'f1_score:{f1_score(Ydev, Ydev_pred, average=None)}')

precision:[0.96703297 0.98809524 0.96666667 0.93877551 0.95294118 0.90123457
 0.97590361 0.96907216 0.96103896]
recall_score:[0.93617021 0.98809524 0.98863636 0.85185185 0.98780488 0.93589744
 0.97590361 0.98947368 0.93670886]
f1_score:[0.95135135 0.98809524 0.97752809 0.89320388 0.97005988 0.91823899
 0.97590361 0.97916667 0.94871795]


上のように，3番目（`livedoor-homme`）の再現率が低い．

実際訓練データについても`livedoor-homme`の要素数が最も少ないため，学習が進まなかったのだと考えられる．

In [123]:
for i in range(len(VY.classes_)):
    print(f'{i}: {len(np.where(Ytrain == i)[0])}\t{VY.classes_[i]}')

0: 685	dokujo-tsushin
1: 694	it-life-hack
2: 697	kaden-channel
3: 405	livedoor-homme
4: 694	movie-enter
5: 687	peachy
6: 712	smax
7: 714	sports-watch
8: 614	topic-news


そこで，`class_weight='balanced'`を指定して同じように学習すると性能が上がると考えたが，以下のようにテストデータの性能は下がっている．

(`{'alpha': 6.246369486906041e-07, 'l1_ratio': 0.0005504654723293009}`)

|       | accuracy | precision | recall | f1_measure |
| ----- | -------- | --------- | ------ | ---------- |
| train | 1.000      | 1.000       | 1.000    | 1.000        |
| dev   | 0.973    | 0.972     | 0.969  | 0.970      |
| test  | 0.966    | 0.965     | 0.961  | 0.963      |

また，混同行列は，３番め目の分類についてごく僅かに改善の傾向が見られるが，誤差の範囲である．

In [147]:
confusion_matrix(Ydev, Ydev_pred)

array([[91,  0,  0,  1,  0,  2,  0,  0,  0],
       [ 0, 84,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 88,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  0, 48,  2,  2,  0,  0,  0],
       [ 0,  0,  0,  0, 81,  1,  0,  0,  0],
       [ 1,  0,  0,  1,  2, 74,  0,  0,  0],
       [ 0,  0,  2,  0,  0,  0, 81,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 94,  1],
       [ 0,  0,  0,  0,  0,  1,  0,  2, 76]])

## TRIAL

In [2]:
#用いたライブラリ．　一部使用していないものもある

from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_validate, StratifiedKFold

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler

import optuna
from collections import defaultdict
import numpy as np
import re

import neologdn
import umap
from sklearn.decomposition import TruncatedSVD

In [69]:
# データ読込
import json

with open('livedoor.json') as fi:
    D = json.load(fi)
    
Dtrain = D['train']
Ddev = D['dev']
Dtest = D['test']

In [23]:
# 標準化
# 今回はあまり効果が出ないので用いていない
ss_X = StandardScaler()
Xtrain_ = ss_X.fit_transform(Xtrain)
Xdev_ = ss_X.transform(Xdev)
Xtest_ = ss_X.transform(Xtest)

In [3]:
import MeCab
tagger = MeCab.Tagger('-Owakati')

# mecab-ipadic-neologdを使っても性能が出なかった
# neologd = MeCab.Tagger('-d /usr/lib/mecab/dic/mecab-ipadic-neologd')
# neologd.parse('') 

def tokenize(s):
    return tagger.parse(s).split()

def token2vec(token):
    vec = defaultdict(int)
    for sentence in token:
        for word in sentence:
                vec[word] += 1
    return vec

In [22]:
urlre=re.compile(r'(http|https)://([-\w]+\.)+[-\w]+(/[-\w./?%&=]*)?')
symbolre = re.compile('[，．、。]')
# numre = re.compile(r'\d+')

def mytokenize(d):
    s=''.join(d)
    token = []
    s=s.replace('\u3000','')
    neologdn.normalize(s)
    s=urlre.sub("", s)
    s=symbolre.sub(" ", s)
#     s=numre.sub('0', s)
    node = tagger.parseToNode(s)
    while node:
        features = node.feature.split(',')
        pos = features[0]
#         pos_sub1 = features[1]
        base = features[6]
        if node.surface == '':
            node = node.next
            continue
        if pos in ['名詞', '動詞', '形容詞', '連体詞', '副詞', '感動詞', '記号']: # and pos_sub1 not in  ['非自立', '接尾']:
            if base == "*":
                token.append(node.surface)
            else:
                token.append(base)

        node = node.next

    return token

In [88]:
# vector化 TFIDF:単語の重要度によるベクトル化． 単語の出現頻度と逆文書頻度（単語の希少さ）の積
# min_df=3 : 出現数3未満の語彙は除外
# max_df=0.7 : 70%の文書で出現する語彙は除外
vectorizer = TfidfVectorizer(analyzer=mytokenize,min_df=3, max_df=0.7, norm='l2', sublinear_tf=True)
vectorizer.fit([[d['title']]+d['text'] for d in Dtrain])

# 用いるデータ
# titleとtextを結合してベクトル化
Xtrain_tfidf = vectorizer.transform([[d['title']]+ d['text'] for d in Dtrain])
Xdev_tfidf =vectorizer.transform([[d['title']]+ d['text'] for d in Ddev])
Xtest_tfidf = vectorizer.transform([[d['title']]+ d['text'] for d in Dtest])

# 分類
VY = preprocessing.LabelEncoder()
Ytrain = VY.fit_transform([data['source'] for data in Dtrain])
Ydev = VY.transform([data['source'] for data in Ddev])
Ytest = VY.transform([data['source'] for data in Dtest])

In [695]:
# opt1(不採用)
# 訓練データの交差検証によるチューニング
Xtrain_opt = Xtrain_tfidf
Xdev_opt = Xdev_tfidf

def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-10, 1e-2)
    l1_ratio = trial.suggest_loguniform('l1_ratio', 1e-10, 1e-2)
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    clf = SGDClassifier(loss='log', alpha=alpha, l1_ratio = l1_ratio)
    scores=cross_validate(clf, X=Xtrain_opt,y=Ytrain, scoring='f1_macro',cv=skf)
    return scores['test_score'].mean()

In [24]:
# opt2
# 検証データを用いたパラメータチューニング
Xtrain_opt = Xtrain_tfidf
Xdev_opt = Xdev_tfidf
def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-10, 1e-2)
    l1_ratio = trial.suggest_loguniform('l1_ratio', 1e-10, 1)
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    clf = SGDClassifier(loss='log', alpha=alpha, l1_ratio = l1_ratio)
    clf.fit(Xtrain_opt, Ytrain)
    Ydev_pred = clf.predict(Xdev_opt)
    return f1_score(Ydev, Ydev_pred, average="macro")

In [ ]:
# 1000回のトライアルで最適なパラメータを採用
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)
params=study.best_params

In [60]:
# パラメータは固定して100回の施行で検証データのf1scoreが最大となるモデルを採用
best_score=0
for i in range(100):
    clf = SGDClassifier(loss='log')
    clf.set_params(**params)
    clf.fit(Xtrain_tfidf, Ytrain)
    Ydev_pred=clf.predict(Xdev_tfidf)
    if best_score< f1_score(Ydev, Ydev_pred, average="macro"):
        best_model = clf
        best_score = f1_score(Ydev, Ydev_pred, average="macro")
print(best_score)
clf = best_model

0.9684165110263153


In [ ]:
#model をpickle化
import pickle

with open('SGD_best.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [65]:
# 結果の出力(検証データ最大)
Ytrain_prev=clf.predict(Xtrain_tfidf)
Ydev_pred=clf.predict(Xdev_tfidf)
Ytest_pred=clf.predict(Xtest_tfidf)
print(params)
print(f'train: {clf.score(Xtrain_tfidf, Ytrain)}')
print(f'precision:{precision_score(Ytrain, Ytrain_prev, average="macro")}')
print(f'recall_score:{recall_score(Ytrain, Ytrain_prev, average="macro")}')
print(f'f1_score:{f1_score(Ytrain, Ytrain_prev, average="macro")}')
print(f'dev: {clf.score(Xdev_tfidf, Ydev)}')
print(f'precision:{precision_score(Ydev, Ydev_pred, average="macro")}')
print(f'recall_score:{recall_score(Ydev, Ydev_pred, average="macro")}')
print(f'f1_score:{f1_score(Ydev, Ydev_pred, average="macro")}')
print(f'test: {clf.score(Xtest_tfidf, Ytest)}')
print(f'test precision:{precision_score(Ytest, Ytest_pred, average="macro")}')
print(f'test recall_score:{recall_score(Ytest, Ytest_pred, average="macro")}')
print(f'test f1_score:{f1_score(Ytest, Ytest_pred, average="macro")}')

{'alpha': 0.003808901322441786, 'l1_ratio': 5.332697964899795e-08}
train: 1.0
precision:1.0
recall_score:1.0
f1_score:1.0
dev: 0.9715061058344641
precision:0.9709592982837232
recall_score:0.9669368023926308
f1_score:0.9684165110263153
test: 0.9701492537313433
test precision:0.968393445978932
test recall_score:0.9652803286692152
test f1_score:0.9665045737809737

train: 1.0
dev: 0.9715061058344641
test: 0.9701492537313433


In [612]:
# svdによる次元削減（不採用）
svd=TruncatedSVD(n_components=1000)
svd.fit(Xtrain_tfidf)

Xtrain_svd=svd.transform(Xtrain_tfidf)
Xdev_svd=svd.transform(Xdev_tfidf)
Xtest_svd=svd.transform(Xtest_tfidf)

In [62]:
# umapによる次元削減（不採用）
um=umap.UMAP()
um.fit(Xtrain_tfidf)

Xtrain_um=um.transform(Xtrain_tfidf)
Xdev_um=um.transform(Xdev_tfidf)
Xtest_um=um.transform(Xtest_tfidf)

---

In [124]:
# opt2
# 検証データを用いたパラメータチューニング
Xtrain_opt = Xtrain_tfidf
Xdev_opt = Xdev_tfidf
def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-10, 1e-2)
    l1_ratio = trial.suggest_loguniform('l1_ratio', 1e-10, 1)
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    clf = SGDClassifier(loss='log', alpha=alpha, l1_ratio = l1_ratio, class_weight='balanced')
    clf.fit(Xtrain_opt, Ytrain)
    Ydev_pred = clf.predict(Xdev_opt)
    return f1_score(Ydev, Ydev_pred, average="macro")

In [ ]:
# 1000回のトライアルで最適なパラメータを採用
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)
params=study.best_params

In [126]:
print(params)

{'alpha': 6.246369486906041e-07, 'l1_ratio': 0.0005504654723293009}


In [142]:
# パラメータは固定して100回の施行で検証データのf1scoreが最大となるモデルを採用
best_score=0
for i in range(100):
    clf = SGDClassifier(loss='log', class_weight='balanced')
    clf.set_params(**params)
    clf.fit(Xtrain_tfidf, Ytrain)
    Ydev_pred=clf.predict(Xdev_tfidf)
    if best_score< f1_score(Ydev, Ydev_pred, average="macro"):
        best_model = clf
        best_score = f1_score(Ydev, Ydev_pred, average="macro")
print(best_score)
clf = best_model

0.9701996303242613


In [143]:
#model をpickle化
import pickle

with open('SGD_best_balanced.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [146]:
# 結果の出力(検証データ最大)
Ytrain_prev=clf.predict(Xtrain_tfidf)
Ydev_pred=clf.predict(Xdev_tfidf)
Ytest_pred=clf.predict(Xtest_tfidf)
print(params)
print(f'train: {clf.score(Xtrain_tfidf, Ytrain)}')
print(f'precision:{precision_score(Ytrain, Ytrain_prev, average="macro")}')
print(f'recall_score:{recall_score(Ytrain, Ytrain_prev, average="macro")}')
print(f'f1_score:{f1_score(Ytrain, Ytrain_prev, average="macro")}')
print(f'dev: {clf.score(Xdev_tfidf, Ydev)}')
print(f'precision:{precision_score(Ydev, Ydev_pred, average="macro")}')
print(f'recall_score:{recall_score(Ydev, Ydev_pred, average="macro")}')
print(f'f1_score:{f1_score(Ydev, Ydev_pred, average="macro")}')
print(f'test: {clf.score(Xtest_tfidf, Ytest)}')
print(f'test precision:{precision_score(Ytest, Ytest_pred, average="macro")}')
print(f'test recall_score:{recall_score(Ytest, Ytest_pred, average="macro")}')
print(f'test f1_score:{f1_score(Ytest, Ytest_pred, average="macro")}')

{'alpha': 6.246369486906041e-07, 'l1_ratio': 0.0005504654723293009}
train: 0.9998305659098611
precision:0.9998193315266486
recall_score:0.9998439450686641
f1_score:0.9998315099623546
dev: 0.9728629579375848
precision:0.9719748031952378
recall_score:0.9689888263514056
f1_score:0.9701996303242613
test: 0.966078697421981
test precision:0.9647403127029965
test recall_score:0.9610233444545434
test f1_score:0.9626538472838951
